In [1]:
import os
import sys
import yaml

import tensorflow as tf
from tensorflow.keras.utils import Progbar
import tensorflow.keras as K
import librosa
import numpy as np
import glob

2024-06-07 15:40:38.702788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

from model.dataset import Dataset
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer
from model.fp.nnfp import get_fingerprinter


In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
tf.__version__

'2.15.0'

# Functions definition

In [6]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    return m_pre, m_specaug, m_fp

def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

@tf.function
def test_step(X, m_pre, m_fp):
    """ Test step used for mini-search-validation """
    X = tf.concat(X, axis=0)
    feat = m_pre(X)  # (nA+nP, F, T, 1)
    m_fp.trainable = False
    emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)
    emb_f_postL2 = tf.math.l2_normalize(emb_f, axis=1)
    emb_gf = m_fp.div_enc(emb_f)
    emb_gf = tf.math.l2_normalize(emb_gf, axis=1)
    return emb_f, emb_f_postL2, emb_gf # f(.), L2(f(.)), L2(g(f(.))

@tf.function
def predict(X, m_pre, m_fp):
    """ 
    Test step used for mini-search-validation 
    X -> (B,1,8000)
    """
    feat = m_pre(X)  # (nA+nP, F, T, 1)
    m_fp.trainable = False
    emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)
    emb_f_postL2 = tf.math.l2_normalize(emb_f, axis=1)
    emb_gf = m_fp.div_enc(emb_f)
    emb_gf = tf.math.l2_normalize(emb_gf, axis=1)
    return emb_f, emb_f_postL2, emb_gf # f(.), L2(f(.)), L2(g(f(.))

# Load latest checkpoint

In [7]:
checkpoint_name_dir:str = "./logs/CHECKPOINT_BSZ_120"#"CHECKPOINT"   # string
checkpoint_index:int = None  # int
config:str = "default"   

In [8]:
cfg = load_config(config)


cli: Configuration from ./config/default.yaml


In [9]:
m_pre, m_specaug, m_fp = build_fp(cfg)

In [10]:

checkpoint = tf.train.Checkpoint(m_fp)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_name_dir))

# Load data

In [11]:
audio_path = "/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/db/000/000134.wav"

In [12]:
audio,fs = librosa.load(audio_path, mono=True, sr=8000)

In [13]:
dataset = Dataset(cfg)

In [14]:
test_db_ds = dataset.get_test_dummy_db_ds()

In [16]:
len(test_db_ds)

0

In [28]:
X = test_db_ds.__getitem__(0)

In [42]:
X[0][0][None,:].shape

(1, 1, 8000)

In [30]:
Y = tf.concat(X, axis=0)

In [31]:
Y.shape

TensorShape([125, 1, 8000])

# Model Predict

In [43]:
emb_f, emb_f_postL2, emb_gf = predict(X[0][0][None,:], m_pre, m_fp)

In [45]:
emb_gf.shape

TensorShape([1, 128])

In [46]:
emb_gf

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.07539804, -0.16872996, -0.12115411, -0.01941191, -0.0148505 ,
         0.08560746, -0.0388802 ,  0.11214361,  0.02991246,  0.12607488,
         0.00300827,  0.06731218, -0.01309047,  0.03161237, -0.12762582,
         0.04633382,  0.11228829,  0.01818596, -0.09356156, -0.08653133,
        -0.15511537, -0.10948697, -0.00143263,  0.06219649,  0.00831405,
         0.04150734, -0.0522717 ,  0.0057195 , -0.0325827 , -0.03062948,
         0.03439471,  0.03571294, -0.08395135,  0.01864311,  0.03021016,
         0.02572118, -0.00137527,  0.01199023, -0.21145529, -0.03078636,
        -0.09409375,  0.08832733,  0.02622688, -0.0805587 , -0.04767458,
        -0.07374059, -0.07926158, -0.00388286,  0.06218813, -0.09946233,
        -0.14341326,  0.10287136,  0.0331257 ,  0.02296653,  0.05973741,
        -0.1981869 ,  0.04241665, -0.18951239,  0.12496328, -0.08082911,
        -0.00096449, -0.03653152, -0.02718567, -0.11009899,  0.1064317 ,
 

# System Audio Fingerprinting

### Load Reference Dataset and Extract Embedded

In [50]:
files_db_dir = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-dummy-db-100k-full/fma_full'
files_query_dir = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-dummy-db-100k-full/fma_full'

In [47]:
d = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/test_ids_icassp2021.npy'

In [51]:

y = np.load(d)

In [49]:
y

array([14655, 21662,  7594, ..., 15722,  1097,  4538])

In [52]:
files = glob.glob(os.path.join(files_db_dir,"**/*.wav"))

In [53]:
len(files)

93458

### Create Vector Database

### Search

# Ver o ficheiro pickle

In [1]:
root_out = '/mnt/dataset/features'

In [2]:
import pickle

In [3]:
# 1. Carregar o arquivo pickle
file_path = root_out + '/002/002000.pkl'

data = [] 
with open(file_path, "rb") as f:
    data = pickle.load(f)

2024-04-28 21:51:32.126885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# 2. Explorar os dados
print("Tipo de Dados:", type(data))

Tipo de Dados: <class 'tensorflow.python.framework.ops.EagerTensor'>


In [5]:
print(data)

tf.Tensor(
[[ 0.01796628 -0.10318357 -0.04985833 ... -0.15665779 -0.05464523
  -0.08112149]
 [ 0.01028875 -0.03953161  0.26217332 ...  0.03502701  0.00542068
   0.00784411]
 [ 0.0317118   0.0627202  -0.15309468 ...  0.12765522 -0.05246528
  -0.05398506]
 ...
 [ 0.03320696 -0.11092483 -0.05904498 ... -0.1455077  -0.18099743
   0.14961286]
 [ 0.01825167  0.01535849 -0.12124566 ...  0.03751192  0.03870636
   0.07288236]
 [ 0.00894083  0.02610302  0.0102381  ...  0.09504692  0.05563493
  -0.07567749]], shape=(549, 128), dtype=float32)


In [6]:
# 2. Converter o tensor TensorFlow para um array numpy
numpy_data = data.numpy()

# 3. Agora você pode analisar o array numpy como desejar
print("Tipo de Dados (Numpy):", type(numpy_data))
print("Forma do Array:", numpy_data.shape)
print("Dados:", numpy_data)

Tipo de Dados (Numpy): <class 'numpy.ndarray'>
Forma do Array: (549, 128)
Dados: [[ 0.01796628 -0.10318357 -0.04985833 ... -0.15665779 -0.05464523
  -0.08112149]
 [ 0.01028875 -0.03953161  0.26217332 ...  0.03502701  0.00542068
   0.00784411]
 [ 0.0317118   0.0627202  -0.15309468 ...  0.12765522 -0.05246528
  -0.05398506]
 ...
 [ 0.03320696 -0.11092483 -0.05904498 ... -0.1455077  -0.18099743
   0.14961286]
 [ 0.01825167  0.01535849 -0.12124566 ...  0.03751192  0.03870636
   0.07288236]
 [ 0.00894083  0.02610302  0.0102381  ...  0.09504692  0.05563493
  -0.07567749]]


In [10]:
import os

In [7]:
rootDir = '/mnt/dataset/features/002'

In [8]:
data =[] 

In [11]:
for filename in os.listdir(rootDir):
    # Verifique se é um arquivo pickle
    if filename.endswith(".pkl"):
        file_path = os.path.join(rootDir, filename)
        # Abra o arquivo pickle e carregue os dados
        with open(file_path, "rb") as f:
            data = pickle.load(f)
            # Faça algo com os dados, como imprimir ou processar
            print(data)

tf.Tensor(
[[ 0.05014153  0.11020733 -0.03082486 ... -0.0017906  -0.09696002
  -0.02319675]
 [-0.02856958  0.03083866  0.04543855 ... -0.02454284  0.02431647
  -0.03131312]
 [ 0.04264287 -0.12034177  0.11543821 ... -0.14082569 -0.05399149
  -0.09139106]
 ...
 [ 0.04496242 -0.02099428 -0.05330151 ...  0.11928467 -0.02413639
   0.0634053 ]
 [ 0.0254466  -0.05758518  0.11602331 ... -0.05019581  0.08341817
  -0.04364978]
 [-0.06354365  0.0251832  -0.20570211 ...  0.04651435  0.07204343
  -0.02599407]], shape=(160, 128), dtype=float32)
tf.Tensor(
[[ 0.03804016 -0.11052421  0.03811419 ... -0.01081619  0.11171138
   0.03326511]
 [ 0.02801518 -0.06275465 -0.16247521 ... -0.05544231 -0.058114
   0.03354669]
 [-0.02199127 -0.00533282  0.2596038  ... -0.00909631  0.05754183
  -0.06853015]
 ...
 [-0.01849866  0.13758947 -0.08467522 ...  0.02705981  0.05679398
  -0.00444294]
 [ 0.08139803 -0.02847053 -0.12337358 ... -0.08344442 -0.121673
  -0.10574799]
 [ 0.06468755  0.11035293  0.08234246 ... -0.0

In [28]:
#data[0][0]
data.shape

TensorShape([242, 128])

In [25]:
data

<tf.Tensor: shape=(242, 128), dtype=float32, numpy=
array([[ 2.8298629e-02,  1.2362208e-01,  1.3936357e-01, ...,
         4.8584230e-02, -1.3036991e-04, -6.8957508e-02],
       [ 3.2286223e-02,  1.0844775e-01,  1.2271961e-02, ...,
         7.4706696e-02, -3.4895518e-03, -3.9881971e-02],
       [ 6.5511249e-02, -9.6264370e-03,  2.7706477e-01, ...,
        -3.4273524e-02,  1.3123809e-01, -1.0917795e-01],
       ...,
       [-6.5664109e-03,  5.1021900e-02,  2.6181898e-01, ...,
         1.3882338e-01,  1.8668428e-02,  6.6377260e-02],
       [ 3.3185661e-02, -2.5948077e-02,  2.0163156e-02, ...,
         6.7998320e-02, -7.6513742e-03, -9.7504370e-02],
       [ 4.4915315e-02, -5.3707223e-02, -1.9697638e-02, ...,
         9.1537558e-02,  3.3878431e-02, -6.9303162e-02]], dtype=float32)>

In [15]:
#x é emb

In [8]:
import pickle

In [9]:
file = '/mnt/dataset/test/features/000/000134.pkl'
with open(file, "rb") as f:
    x = pickle.load(f)

2024-04-28 22:14:37.274203: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 22:14:37.274374: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 22:14:37.274512: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:
type(x)

tensorflow.python.framework.ops.EagerTensor

## Clean GPU memory

In [12]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()